In [1]:
modules = {}
with open("Day20.txt") as file:
    for line in file.read().splitlines():
        name, outputs = line.split(" -> ")
        if name[0] in "%&":
            modules[name[1:]] = (name[0], outputs.split(", "))
        else:
            modules[name] = ('', outputs.split(", "))

inputs = {
    name: [
        input for input, (_, outputs) 
        in modules.items() if name in outputs
    ] for name in set(modules) | {"rx"}}
init_states = lambda: {
    name: 0 if module == "%" else dict.fromkeys(inputs[name], 0) 
    for name, (module, _) in modules.items() if module}

In [2]:
from collections import deque

def cycle(states, feedback, *extra):
    queue = deque([("button", "broadcaster", 0)])
    while queue:
        source, target, signal = queue.popleft()
        feedback(target, signal, *extra)
        if target not in modules:
            continue
        module, outputs = modules[target]
        if module == "%":
            if signal:
                continue
            states[target] ^= 1
            output_signal = states[target]
        elif module == "&":
            state = states[target]
            state[source] = signal
            output_signal = int(not all(state.values()))
        else:
            output_signal = signal
        queue.extend((target, output, output_signal) for output in outputs)

In [3]:
from math import prod

signals = [0, 0]
def feedback(target, signal, *extra):
    signals[signal] += 1

states = init_states()
for _ in range(1000):
    cycle(states, feedback)
prod(signals)

680278040

In [4]:
from itertools import count
from math import lcm

presses, (input, ) = [], inputs["rx"]
def feedback(target, signal, press):
    if signal and target == input:
        presses.append(press)

states = init_states()
for press in count(1):
    cycle(states, feedback, press)
    if len(presses) == len(states[input]):
        break
lcm(*presses)

243548140870057